## Population Lecture II



### Malthus



Among all the things ever written by social scientists (or
&ldquo;political philosophers&rdquo;, to use a term he would have recognized),
Malthus&rsquo; [&ldquo;Essay on Population&rdquo; (1798)](https://oll.libertyfund.org/titles/malthus-an-essay-on-the-principle-of-population-1798-1st-ed) must be among the most
influential.  You can find the first edition at the link above.
The main argument is spelled out in Chapter 1, which I encourage
you to read.  A key point of the essay begins with [two postulata](https://oll.libertyfund.org/titles/311#Malthus_0195_20):

-   **First:** That food is necessary to the existence of man.
-   **Secondly:** That the passion between the sexes is necessary and
    will remain nearly in its present state.

These lead to a conclusion:

>     Assuming then, my postulata as granted, I say, that the power of
>     population is indefinitely greater than the power in the earth to
>     produce subsistence for man.  Population, when unchecked, increases
>     in a geometrical ratio. Subsistence increases only in an
>     arithmetical ratio. A slight acquaintance with numbers will shew
>     the immensity of the first power in comparison of the second.



#### Malthus meets math



Malthus&rsquo; argument boils down to a mathematical assertion: that
population grows at an exponential rate, while food production
increases only linearly.  We might write this as

$$
       \mbox{Population}_t = \mbox{Population}_0e^{rt},
    $$

where $r$ is the rate of population growth, and $t$ is the number
of years that have passed; and

$$
       \mbox{Food}_t = \mbox{Food}_0 + mt.
    $$

If we lack a &ldquo;slight acquaintance&rdquo; with numbers a graph makes the
point:



In [1]:
from plotly.offline import init_notebook_mode, iplot
import numpy as np
from scipy.optimize import newton

init_notebook_mode(connected=True)

# Play with these constants
## for exponential
P0 = 1
r = 0.02

## for line
F0 = 10
m = 100

food_production = lambda t,m=m: F0+m*t

population = lambda t,r=r: P0*np.exp(r*t)

diff = lambda t,r=r,m=m: food_production(t,m) - population(t,r)

# Point where curves cross

def choose_xaxis(r,m,x0=10):
    """
    Find a range of x so that curves intersect
    """
    tstar = -1
    try:
        while tstar <= 0:
            tstar = newton(lambda t: diff(t,r,m),x0,maxiter=1000)
            x0 = x0*10
        return np.linspace(0,tstar*1.1,100)
    except RuntimeError: # Failed to find root
        return np.linspace(0,1000,100)

In [9]:
import plotly.graph_objs as go
from ipywidgets import interactive, HBox, VBox

T = choose_xaxis(r,m)

data = [dict(name='Food production',
             #visible = False,
             line=dict(color='blue', width=6),
             x = T,
             y = [food_production(t) for t in T]),
        dict(name='Population',
             #visible = False,
             line=dict(color='red', width=6),
             x = T,
             y = [population(t) for t in T])]

layout = {}

#fig = dict(data=data, layout=layout)
#plot(fig)

f = go.FigureWidget(data=data,layout=layout)

def update_pop(r,m):
    T = choose_xaxis(r,m)
    f.data[1].y = [population(t,r) for t in T]
    f.data[0].x = T
    f.data[1].x = T

r_slider = interactive(update_pop,r=(-0.05,0.08,.01),m=(0,200,10))
vb = VBox((f, r_slider))
vb.layout.align_items = 'center'
vb

    'data': [{'line': {'color': 'blue', 'width': 6},
              'name': 'Food…

### Food Production



Malthus thought that population would have to increase at an
   exponential rate.  We&rsquo;ve seen that in fact the rate of population
   increase has been roughly linear over the last six decades. (There
   are no guarantees about the future!)

Malthus also thought that food production (&ldquo;subsistence&rdquo;) could
only increase linearly (&ldquo;arithmetically&rdquo;).  We&rsquo;ll next take a look
to see what the data say about this.



#### Preface



In [10]:
import numpy as np
import wbdata
import cufflinks as cf
cf.go_offline()

def fix_date_index(df):
    idx_vars = df.index.names
    new = df.reset_index()
    new.date = new.date.astype(int)
    return new.set_index(idx_vars)

#### Finding data



Maybe there&rsquo;s something relevant to food production in the WDI
 indicators?



In [11]:
wbdata.get_topic()

  id  value
----  -------------------------------
   1  Agriculture & Rural Development
   2  Aid Effectiveness
   3  Economy & Growth
   4  Education
   5  Energy & Mining
   6  Environment
   7  Financial Sector
   8  Health
   9  Infrastructure
  10  Social Protection & Labor
  11  Poverty
  12  Private Sector
  13  Public Sector
  14  Science & Technology
  15  Social Development
  16  Urban Development
  17  Gender
  18  Millenium development goals
  19  Climate Change
  20  External Debt
  21  Trade

&ldquo;Agriculture & Rural Development&rdquo; sounds relevant.  Let&rsquo;s see what
 variables are available:



In [12]:
wbdata.get_indicator(topic=1)

id                    name
--------------------  ------------------------------------------------------------------------------------------
AG.AGR.TRAC.NO        Agricultural machinery, tractors
AG.CON.FERT.PT.ZS     Fertilizer consumption (% of fertilizer production)
AG.CON.FERT.ZS        Fertilizer consumption (kilograms per hectare of arable land)
AG.LND.AGRI.K2        Agricultural land (sq. km)
AG.LND.AGRI.ZS        Agricultural land (% of land area)
AG.LND.ARBL.HA        Arable land (hectares)
AG.LND.ARBL.HA.PC     Arable land (hectares per person)
AG.LND.ARBL.ZS        Arable land (% of land area)
AG.LND.CREL.HA        Land under cereal production (hectares)
AG.LND.CROP.ZS        Permanent cropland (% of land area)
AG.LND.EL5M.RU.K2     Rural land area where elevation is below 5 meters (sq. km)
AG.LND.EL5M.RU.ZS     Rural land area where elevation is below 5 meters (% of total land area)
AG.LND.FRST.K2        Forest area (sq. km)
AG.LND.FRST.ZS        Forest area (% of land area)

#### Cereal production & land



The great majority of food (as measured by calories) originates in the
 form of cereals; some of these are turned into products directly
 consumed by humans (bread, rice, hominy); others are used to feed
 livestock, which humans subsequently eat.  So what&rsquo;s happened to
 cereal production over the last few decades?



In [13]:
vars = {"AG.PRD.CREL.MT":"Cereal production"}

# Use this to find top producers:
#big_producers = cereals.query('date==2000').squeeze().sort_values(ascending=False)
#big_producers.head(20)

use=["CHN","USA","IND","FRA","RUS","WLD"]

cereals = fix_date_index(wbdata.get_dataframe(vars,country=use)).squeeze().unstack('country')

cereals.iplot(xTitle="Year",yTitle="Metric Tons",
              title="Cereal Production by Top 5 (in 2000)")

#### FAO Food Index



The FAO compiles a series of data meant to measure changes in the
    supply of food of all sorts, including both plant and animal
    sources.  Let&rsquo;s how that&rsquo;s changed over recent decades:



In [14]:
vars = {"AG.PRD.LVSK.XD":"Livestock index",
        "AG.PRD.FOOD.XD":"Food index",
        "AG.PRD.CROP.XD":"Crop index"}

food = fix_date_index(wbdata.get_dataframe(vars,country='WLD'))

food.iplot(xTitle="Year",yTitle="Index")

From this we don&rsquo;t get levels; the indices are normalized so that 2006
 is 100.  Let&rsquo;s compare this with a similarly normalized description of
 world population:



In [15]:
vars = {"AG.PRD.LVSK.XD":"Livestock index",
        "AG.PRD.FOOD.XD":"Food index",
        "AG.PRD.CROP.XD":"Crop index",
        "SP.POP.TOTL":"Population"}

food = fix_date_index(wbdata.get_dataframe(vars,country='WLD'))

food.sort_index(inplace=True)

food['Population'] = food['Population']/food.loc[2006,'Population']*100

food.iplot(xTitle="Year",yTitle="Index")

And here in growth rates:



In [16]:
np.log(food).diff().iplot(xTitle="Year",yTitle="Growth rates")